In [1]:
import os
from datetime import datetime, timedelta
import rioxarray as rxr
import xarray as xr
import rasterio as rio

import get_data_merra

dl_data_dir ='/media/work/T7/merra2-M2T1NXAER-V5.12.4'
int_data_dir = '../data/interim/merra2'

### Download data

In [2]:
get_data_merra.get_data(product = "M2T1NXAER", 
                        output_dir=dl_data_dir,
                        start_date = "2024-01-01",
                        end_date = "2024-12-31"
                        )

### Get annual mean data for India

In [3]:
# Read all the files, crop India, calculate the average over time, and save to a file for easier access
os.makedirs(int_data_dir, exist_ok=True)

filename = 'M2T1NXAER_India_2024.nc'

if not os.path.exists(os.path.join(int_data_dir, filename)):
    # find all data files in the directory
    files = [f for f in os.listdir(dl_data_dir) if f.endswith('.nc4')]
    files.sort()
    files = [os.path.join(dl_data_dir, f) for f in files]

    # open the dataset using xarray
    ds = xr.open_mfdataset(files, combine='by_coords')

    # crop India
    ds = ds.sel(lat=slice(3, 40), lon=slice(64,99))

    # take average over time
    ds = ds.mean(dim='time', keep_attrs=True)

    # save to file
    ds.to_netcdf(os.path.join(int_data_dir, filename))
    print(f"Saved {filename} to {int_data_dir}.")

## Calculate PM2.5

In [4]:
# read the saved file and write CRS
ds = xr.open_dataset(os.path.join(int_data_dir, filename))
ds = ds.rio.write_crs("EPSG:4326")

# calculate PM2.5 as instructed in FAQ (https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/FAQ/)
PM25 = ds.DUSMASS25 + ds.OCSMASS+ ds.BCSMASS + ds.SSSMASS25 + ds.SO4SMASS* (132.14/96.06)

In [5]:
# convert from kg/m3 to ug/m3
PM25 = PM25 * 1e9
PM25.name = 'PM25'
PM25.attrs['units'] = 'ug/m3'

for var in ['DUSMASS25', 'OCSMASS', 'BCSMASS', 'SSSMASS25', 'SO4SMASS']:
    ds[var] = ds[var] * 1e9 
    ds[var].attrs['units'] = 'ug/m3'


# also create variable for sulfate aerosol (ammonium sulfate)
sulfate = ds.SO4SMASS * (132.14/96.06)

## Save to file

In [6]:
out_data_dir = "../data/result/merra/surf_aerosol_massconc"
os.makedirs(out_data_dir, exist_ok=True)

In [7]:
filanames = {
    'PM25': 'Total_PM25',
    'DUSMASS25': 'Dust_PM25',
    'OCSMASS': 'Organic_carbon',
    'BCSMASS': 'Black_carbon',
    'SSSMASS25': 'Seasalt_PM25',
    'sulfate': 'Sulfate_aerosol'
}

# save PM2.5 to a geotiff file
out_file = os.path.join(out_data_dir, filanames['PM25'] + '_2024.tif')
PM25 = PM25.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
PM25.rio.to_raster(out_file)
print(f"Saved PM2.5 annual mean to {out_file}")

# save sulfate aerosol to a geotiff file
out_file = os.path.join(out_data_dir, filanames['sulfate'] + '_2024.tif')
sulfate = sulfate.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
sulfate.rio.to_raster(out_file)
print(f"Saved sulfate aerosol annual mean to {out_file}")

# save each variable in a separate geotiff file
for var in ['DUSMASS25', 'OCSMASS', 'BCSMASS', 'SSSMASS25']:
    # map variable names to long names for output files
    out_file = os.path.join(out_data_dir, filanames[var] + '_2024.tif')
    
    da = ds[var].rio.set_spatial_dims(x_dim="lon", y_dim="lat")
    da.rio.to_raster(out_file)

    print(f"Saved {var} annual mean to {out_file}")

Saved PM2.5 annual mean to ../data/result/merra/surf_aerosol_massconc/Total_PM25_2024.tif
Saved sulfate aerosol annual mean to ../data/result/merra/surf_aerosol_massconc/Sulfate_aerosol_2024.tif
Saved DUSMASS25 annual mean to ../data/result/merra/surf_aerosol_massconc/Dust_PM25_2024.tif
Saved OCSMASS annual mean to ../data/result/merra/surf_aerosol_massconc/Organic_carbon_2024.tif
Saved BCSMASS annual mean to ../data/result/merra/surf_aerosol_massconc/Black_carbon_2024.tif
Saved SSSMASS25 annual mean to ../data/result/merra/surf_aerosol_massconc/Seasalt_PM25_2024.tif


## Fractions

In [ ]:
out_data_dir = "../data/result/merra/surf_aerosol_fractions"

In [9]:
# calculate fraction of sulfate aerosol from total PM2.5

sulfate_fraction = sulfate / PM25 * 100
sulfate_fraction.name = 'Sulfate_precentage'
sulfate_fraction.attrs['units'] = '%'

# save sulfate fraction to a geotiff file
out_file = os.path.join(out_data_dir, 'Sulfate_fraction_2024.tif')
sulfate_fraction = sulfate_fraction.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
sulfate_fraction.rio.to_raster(out_file)
print(f"Saved sulfate fraction annual mean to {out_file}")


Saved sulfate fraction annual mean to ../data/result/merra/surf_aerosol_massconc/Sulfate_fraction_2024.tif
